In [28]:
#!pip3 install Pillow

In [29]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

## Creating Dataset

In [ ]:
def calculate_mu_std(path):
    files = os.listdir(path)
    files.remove(".DS_Store")

    # Calculating mean and std for normalization
    rgb_values = np.concatenate( [Image.open(path+img).getdata() for img in files], axis=0) / 255.

    # rgb_values.shape == (n, 3), 
    # where n is the total number of pixels in all images, 
    # and 3 are the 3 channels: R, G, B.

    # Each value is in the interval [0; 1]

    mu_rgb = np.mean(rgb_values, axis=0)  # mu_rgb.shape == (3,)
    std_rgb = np.std(rgb_values, axis=0)  # std_rgb.shape == (3,)

    return mu_rgb, std_rgb
    

In [39]:
# Loading the data
dataPath = "Data/"

class SneakersDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_csv = pd.read_csv(csv_file)
        self.img_dir = root_dir
        #self.transform = transform

        
        #Define dataset
        self.dataset_dir = os.path.join(os.path.join(os.getcwd(),'load_dataset'), 'root_dir')
        
        self.all_filenames = os.listdir(root_dir)
        self.all_labels = pd.read_csv(csv_file, header=0, index_col=0)
        self.label_meanings = self.all_labels.columns.values.tolist()
        
        
    def __len__(self):
        return len(self.data_csv)-1

    def __getitem__(self, idx): 
        #all normalization and data augmentation steps should be applied to the image before this method returns it.
        '''
        if torch.is_tensor(idx):
            idx = idx.tolist()
        '''
        
        file = os.path.join(self.img_dir, self.data_csv.iloc[idx, 0])
        imagepil = PIL.Image.open(os.path.join(self.img_dir, file)).convert('RGB')
        
        #convert image to Tensor and normalize
        image = utils.to_tensor_and_normalize(imagepil)
        

        label = torch.Tensor(self.all_labels.loc[selected_filename,:].values)
        sample = {'data':image, #preprocessed image, for input into NN
                  'label':label,
                  'img_idx':idx}

        return sample

dataset = SneakersDataset(csv_file='dataInfo.csv', root_dir='data/')
dataLoader = torch.utils.data.DataLoader(dataset, batch_size = 10, shuffle=True, num_workers = 4)
iterator = iter(dataLoader)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'SneakersDataset' on <module '__main__' (built-in)>


KeyboardInterrupt: 

## Creating the model

## Training the model

## Testing the model

## Comparing results

## Conclusion